In [23]:
import math
import numpy as np
from scipy import stats
import pandas as pd
import re
import datetime

# web scraping
from bs4 import BeautifulSoup
import requests
import time
import random
from fake_useragent import UserAgent 

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

ua = UserAgent()
headers = {'user-agent': ua.random}

#### Get Monthly Sales for 2019 & 2020

In [4]:
def get_monthly_sales_df(url, year_str, tbl_num):
    '''
    A function that gets monthly sales of every make and model sold in the us for a specified year

    Parameters
    ----------
    url : url containg monthly sales data, in string format 
    year_str : year in string format
    tbl_num : index number corresponding to the location of the table on the webpage (1st table is at index 0)

    Returns
    -------
    Return a df of monthly US car sales for every make and model, months as columns
    '''
    response = requests.get(url, headers=headers)
    page= response.text
    soup = BeautifulSoup(page, "lxml")
    
    #find car sales data and turn it into a list
    tbl_str = soup.find_all('tbody')[tbl_num].text
    tbl_list = tbl_str.split('\n')
    tbl_list = [i for i in tbl_list if i] 
    
    #create empty time series df for specified year
    date_time_str = year_str + '-01'
    start_date = datetime.datetime.strptime(date_time_str, '%Y-%m')
    index = pd.date_range(start_date, periods=12, freq='m')
    df = pd.DataFrame(index=index)
    
    #fill empty df with monthly sales for all makes and models
    col_name = ''
    idx = 0
    for x in range(0,len(tbl_list)//13):
        col_name = tbl_list[x+idx]
        list_vals = []
        for val in range(x+1,x+13):
            list_vals.append(tbl_list[idx+val])
        df[col_name]=list_vals
        idx = idx+12
    return df

new df called monthly_sales_df

In [11]:
#only monthly sales data for years 2019 and 2020
url = 'https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/' 
df_2019 = get_monthly_sales_df(url, '2019', 2)
url = 'https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model/'
df_2020 = get_monthly_sales_df(url, '2020', 0)
monthly_sales_df = pd.concat((df_2019, df_2020))

pickle df of monthly sales for 2019 and 2020 to data folder

In [14]:
monthly_sales_df.tail()

,Acura ILX,Acura MDX,Acura NSX,Acura RDX,Acura RLX,Acura TLX,Alfa Romeo 4C,Alfa Romeo Giulia,Alfa Romeo Stelvio,Audi A3,...,Ford Mustang Mach E,Genesis GV80,Kia K5,Kia Seltos,Land Rover Defender,Mazda 2,Tesla Model Y,Volkswagen Atlas Sport,Volkswagen ID.4,Volvo 40 Series
2020-08-31,"1,331","5,165",3,"5,057",118,"1,973",7,683,886,796,...,0,0,"3,631","5,314",405,0,"10,909",0,0,0
2020-09-30,"1,377","4,920",10,"4,664",122,"1,848",7,740,960,863,...,0,0,"5,763","5,613",439,0,"11,818",0,0,0
2020-10-31,"1,523","4,732",22,"5,022",126,"2,365",2,864,"1,189","1,094",...,0,0,"7,528","5,542","2,429",0,"1,687","3,737",0,0
2020-11-30,"1,141","4,491",9,"4,155",81,"2,014",2,709,977,899,...,0,58,"7,437","5,122","1,995",0,"1,386","3,070",0,0
2020-12-31,"1,417","6,040",14,"5,820",72,"2,285",3,987,"1,359","1,250",...,3,"1,459","6,029","6,107","2,776","3,256","1,928","4,271",0,0


In [15]:
monthly_sales_df.to_pickle('../data/monthly_sales_df.pkl')

#### Get All Model Yearly Sales for 2005-2020

In [32]:
def get_model_sales_df(url, year):
    '''
    A function that gets all year end model sales for a specified year

    Parameters
    ----------
    url : url containg year end sales data for every make and model sold in the US, in string format
    year : year in string format

    Returns
    -------
    Return a df of yearly US car sales for every make and model 
    columns = Model, Year, and Total_Sales
    '''
    #read url page into list of pandas dfs 
    response = requests.get(url, headers=headers)
    page = response.text
    df_list = pd.read_html(page)
    
    #find correct df based on number of data frames on url page and 
    if len(df_list) == 1 or year == '2019':
        df = pd.DataFrame(df_list[0])
    elif len(df_list) == 2:
        df = pd.DataFrame(df_list[1])
    else:
        df = pd.DataFrame(df_list[len(df_list)-1])
    
    #special case for 2020 data because sales data is in monthly sales format for each model
    if year == '2020':  
        soup = BeautifulSoup(page, "lxml")
    
        #find car sales data and turn it into a list
        tbl_str = soup.find_all('tbody')[0].text
        tbl_list = tbl_str.split('\n')
        tbl_list = [i for i in tbl_list if i] 
        
        columns = ('Model', 'Year', 'Total_Sales')
        df = pd.DataFrame(columns=columns)

        #fill empty df with monthly sales for all makes and models
        model_name = ''
        idx = 0
        index = 0
        for x in range(0,len(tbl_list)//13):
            model_name = tbl_list[x+idx]
            year_sum = 0
            for val in range(x+1,x+13):
                month_total = tbl_list[idx+val].replace(',','')
                month_total = int(month_total)
                year_sum = year_sum + month_total
            #df1 = pd.DataFrame([model_name, year_sum], columns = ['Model', year])
            df = df.append(pd.DataFrame({'Model': model_name, 'Year': year, 'Total_Sales': year_sum}, index=[index]), 
                           ignore_index=True)
            index = index+1
            #df.append(df1)
            idx = idx+12
        return df
    
    #special cases for finding the location of the total sales based on years
    if year == '2012':
        df = df.iloc[:, [2,3]]
    elif year == '2005':
        df = df.iloc[:, [1,3]]
    elif year in ['2017','2018', '2019', '2020']:
        df = df.iloc[:, [0,4]]
    else:
        df = df.iloc[:, [1,2]]
    
    #add columns to df and year column
    df.columns = ['Model', 'Total_Sales']
    df['Year'] = year
    
    #* indicate further breakdown of sum totals, overall totals be removed further down when duplicates are removed
    #other symbols refer to subnotes in the tables and are not apart of model names
    symbols = ['*', '²', '¹', '^', '†', '‡']
    for s in symbols:
        df['Model'] = df['Model'].str.replace(s,'')

    #clean model and Total_Sales column
    df.dropna(subset=['Model'], inplace=True)
    df['Model'] = df['Model'].str.rstrip()
    df['Model'] = df['Model'].str.lstrip()
    df['Total_Sales'] = df['Total_Sales'].apply(pd.to_numeric, errors='coerce') #to numeric
    
    #remove first in set of duplicates b/c first is a sum of a car and the hybrid model
    df.drop_duplicates(subset='Model', keep='last', inplace=True)
    
    return df

In [139]:
#dictionary of all urls containing all year end sales data with year as the key
url_dict = {'2006': 'https://www.goodcarbadcar.net/2006-usa-auto-sales-rankings-by-mode/',
            '2007': 'https://www.goodcarbadcar.net/usa-2007-vehicle-sales-rankings-by-mode/',
            '2008': 'https://www.goodcarbadcar.net/2008-america-auto-sales-rankings-by-mode/',
            '2009': 'https://www.goodcarbadcar.net/usa-auto-sales-rankings-by-model-2009/',
            '2010': 'https://www.goodcarbadcar.net/2010-america-auto-sales-rankings-by-mode/',
            '2011': 'https://www.goodcarbadcar.net/top-268-best-selling-vehicles-2011-year/',
            '2012': 'https://www.goodcarbadcar.net/2012-usa-auto-sales-rankings-by-model7/',
            '2013': 'https://www.goodcarbadcar.net/usa-vehicle-sales-rankings-by-model-december-2013-year-end/',
            '2014': 'https://www.goodcarbadcar.net/usa-all-cars-sales-figures-2014-december-year-end/',
            '2015': 'https://www.goodcarbadcar.net/usa-car-sales-by-model-2015-year-end-december/',
            '2016': 'https://www.goodcarbadcar.net/usa-2016-vehicle-sales-by-model-manufacturer-brand/',
            '2017': 'https://www.goodcarbadcar.net/december-2017-year-end-u-s-passenger-car-sales-rankings-top-171-best-selling-cars-america-every-car-ranked/',
            '2018': 'https://www.goodcarbadcar.net/december-2018-the-best-selling-vehicles-in-america-every-vehicle-ranked/',
            '2019': 'https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/',
            '2020': 'https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model/'
           }

url = 'https://www.goodcarbadcar.net/2006-usa-auto-sales-rankings-by-mode/'
yearly_sales_df= get_model_sales_df(url = url, year = '2005') #new yearl_sales_df to store all yearly sales data

years = ['2005']
#loop through dictionary with urls and stack data frames 
for key, value in url_dict.items():
    years.append(key)
#     print(years)
    df = get_model_sales_df(url = value, year = key) #get df of total sales
    yearly_sales_df = pd.concat([yearly_sales_df, df], axis=0)

print(yearly_sales_df.tail())

               Model Total_Sales  Year
299  Volvo 60-Series       15729  2020
300  Volvo 90-Series        3195  2020
301       Volvo XC40       23778  2020
302       Volvo XC60       32078  2020
303       Volvo XC90       34251  2020


In [140]:
#remove rows with no sales
yearly_sales_df = yearly_sales_df[yearly_sales_df.Total_Sales != 0]
yearly_sales_df.reset_index(drop=True, inplace=True) #reset index after stacking dfs

#Clean total sales dataframe by removing rows containing certain strings
remove_strings = ["Market",'Total','Family','Brand','Passenger Cars, SUVs, Crossovers','Minivans','Pickup Trucks',
                 'Commercial Vans', 'COMPANY', 'MOTOR', 'GROUP', 'AMERICAN', 'AUTOMOBILES', 'JAGUAR', 'DAIMLER']
for string in remove_strings:
    yearly_sales_df = yearly_sales_df[~yearly_sales_df.Model.str.contains(string)]
    
print(yearly_sales_df.shape)
print(yearly_sales_df.tail())
yearly_sales_df.to_pickle('../data/yearly_sales_df.pkl')

(4394, 3)
                Model Total_Sales  Year
4512  Volvo 60-Series       15729  2020
4513  Volvo 90-Series        3195  2020
4514       Volvo XC40       23778  2020
4515       Volvo XC60       32078  2020
4516       Volvo XC90       34251  2020


### Get all Make and Model Specs for Every Car Year Since 2005

In [37]:
def get_brand_links(org_url):
    '''
    A function that gets all links to makes and models for every car brand

    Parameters
    ----------
    url : url containg links to all brands

    Returns
    -------
    Return a dictionary of links to every make and model for every brand 
    Key = brand, value = brand url
    '''
    response = requests.get(org_url, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    brand_link_dict = dict() #empty dict for all urls
    link_list = soup.find_all('li') #get all list objects on the url page
    #loop through all list objects, starting at index 7 where the brand links start
    for x in link_list[7:]:
        brand = x.text
        link = 'https://www.carspecs.us' + x.find('a')['href']
        brand_link_dict[brand] = link
        time.sleep(0.5) #pause
    return brand_link_dict

In [38]:
def get_model_links(brand, url):
    '''
    A function that gets all links to every make, model, and year greater than 2005 ever sold by a specified car brand

    Parameters
    ----------
    url : url containg links to every make and model ever sold by a specified car brand, string format
    brand : brand name in string format

    Returns
    -------
    Return a dataframe of links to every make, model, and year for brand 
    columns = Model, Year, and Model url 
    '''
    #load url page with all car models for specified brand url
    response = requests.get(url, headers=headers) #random user agent
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #find first div with all model links
    div_list = soup.find_all('div', class_='pure-u-1 pure-u-md-1-2')
    
    #new df to store all makes and models for all years past 2004
    columns = ('Model', 'Year', 'Model_url')
    df = pd.DataFrame(columns=columns)
    
    #loop through all model links for specified brand 
    for x in div_list[1].find_all('li'):
        model = brand + ' ' + x.text #brand and model name
        link = org_url + x.find('a')['href'] #model url
        
        #load model url to get list of model year urls
        response = requests.get(link)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        year_links = soup.find_all('li') #list of all year urls for specified model
        
        #loop through all year links starting at index 7
        idx = 7
        index = 0
        for y in year_links[7:]:
            year = year_links[idx].text
            if year in years: #only add year links that are 2005 to 2020
                model_link = 'https://www.carspecs.us' + year_links[idx].find('a')['href'] #add orginal url to string
                df = df.append(pd.DataFrame({'Model': model, 'Year': year, 'Model_url': model_link}, index=[index]), 
                                ignore_index=True)
                index = index+1
                idx = idx+1
        time.sleep(0.5) #pause 
            
    return df

In [39]:
def get_model_specs(url,head):
    '''
    A function that gets all specs for a specified make, model, and year of a car

    Parameters
    ----------
    url : url containg links to every year make and model ever sold by a specified car brand, string format
    hear : header for a page request, in dictioanry format

    Returns
    -------
    Return a list of model specs 
    '''
    response = requests.get(url, headers=head)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    #set all values to nan
    doors =passengers =speed =horsepower =drive =engine =tank =volume =length =width =height = wheelbase =float('NaN')
    mpg = ''
    
    #try to find price
    try:
        price = soup.find(text=re.compile('starting from'))
        price = price.findNext().text
    
    except:
        price = float('NaN')
    
    #list of all divs containg car specs
    div_list = soup.find('div', class_='car-details').find_all('div')

    #loop through all divs and search for strings to find certain car specs and assign to varibales if found
    for div in div_list[1:]:
        #create list, index 0 containg the spec, and index 1 containg the spec value
        spec_list = div.text.split('\n')
        spec_list = [i for i in spec_list if i] #remove empty values in list
        if spec_list:
            if 'RPM' not in spec_list[0]:
                if 'Passenger Doors' in spec_list[0]:
                    doors = int(spec_list[-1])
                if 'Passenger Capacity' in spec_list[0]:
                    passengers = int(spec_list[-1])
                if 'mph' in spec_list[0]:
                    speed = spec_list[-1]
                if 'Horsepower' in spec_list[0]:
                    horsepower = spec_list[-1]
                if 'Drive type' in spec_list[0]:
                    drive = spec_list[-1]
                if 'combined' in spec_list[0]:
                    mpg = spec_list[-1]
                if 'Combined' in spec_list[0]:
                    mpg = spec_list[-1]
                if 'Engine type' in spec_list[0]:
                    engine = spec_list[-1]
                    engine = engine.replace('\t', '')
                if 'tank capacity' in spec_list[0]:
                    tank = spec_list[-1]
                if 'EPA interior' in spec_list[0]:
                    volume = spec_list[-1]
                if 'Length' in spec_list[0]:
                    length = spec_list[-1]
                if 'Width' in spec_list[0]:
                    width = spec_list[-1]
                if 'Height' in spec_list[0]:
                    height = spec_list[-1]
                if 'Wheelbase' in spec_list[0]:
                    wheelbase = spec_list[-1]
        
        #if mpg not found in spec lists, find combined mpg by averaged highway and city mpg by searching for strings
        if mpg == '':
            mpg = soup.find(text=re.compile('highway mpg'))
            try:
                mpg_list = mpg.split('/ ')
                city = mpg_list[0].lstrip('\r\n ')
                highway = mpg_list[1]
                mpg = (int(city[0:2])+int(highway[0:2]))/2
            except:
                mpg = float('NaN') #return nan if not found
                
    #time.sleep(.1+.5*random.random()) #random pause 
    
    return [price, doors, passengers, speed, horsepower, drive, mpg, engine, tank, volume, length, width, height,wheelbase]

#### Get a dictionary of links to all models of a car brand 

In [40]:
org_url = 'https://www.carspecs.us/'
brand_links = get_brand_links(org_url)

#### Get a dictionary of links to all years of a specific make and model 

In [41]:
columns = ('Model', 'Year', 'Model_url')
model_links_df = pd.DataFrame(columns=columns)
for key, value in brand_links.items():
    df = get_model_links(key, brand_links[key])
    model_links_df = model_links_df.append(df, ignore_index=True)
print(model_links_df)

           Model  Year                                     Model_url
0      Acura ILX  2020   https://www.carspecs.us/cars/2020/acura/ilx
1      Acura ILX  2019   https://www.carspecs.us/cars/2019/acura/ilx
2      Acura ILX  2018   https://www.carspecs.us/cars/2018/acura/ilx
3      Acura ILX  2017   https://www.carspecs.us/cars/2017/acura/ilx
4      Acura ILX  2016   https://www.carspecs.us/cars/2016/acura/ilx
...          ...   ...                                           ...
7020  Volvo XC90  2009  https://www.carspecs.us/cars/2009/volvo/xc90
7021  Volvo XC90  2008  https://www.carspecs.us/cars/2008/volvo/xc90
7022  Volvo XC90  2007  https://www.carspecs.us/cars/2007/volvo/xc90
7023  Volvo XC90  2006  https://www.carspecs.us/cars/2006/volvo/xc90
7024  Volvo XC90  2005  https://www.carspecs.us/cars/2005/volvo/xc90

[7025 rows x 3 columns]


#### Create new df containging specs of every make, model, and year of a car

In [45]:
columns = ('Model', 'url',  'drive','engine','price','doors', 'passengers', 'speed_sec', 'horsepower_hp', 'mpg',  
           'tank_gal', 'volume_cuft', 'length_in', 'width_in', 'height_in','wheelbase_in')
model_spec_df = pd.DataFrame(columns=columns) 

#loop through all rows to get all links for every make, model, and year and send to function to get specs
#append specs to model spec dataframe
idx = 0
for index, row in model_links_df.iterrows():
    specs = get_model_specs(row[2],headers)
    model_spec_df = model_spec_df.append(pd.DataFrame({'Model': row[0],'Year':row[1], 'url': row[2], 'drive': specs[5], 'engine': specs[7],
                                                       'price': specs[0],'doors': specs[1],'passengers': specs[2],'speed_sec': specs[3], 
                                                       'horsepower_hp': specs[4],'mpg': specs[6],
                                                       'tank_gal': specs[8], 'volume_cuft': specs[9], 'length_in': specs[10],
                                                      'width_in': specs[11],'height_in': specs[12],'wheelbase_in':specs[13]}, index=[idx]), 
                                                         ignore_index=True)
    idx = idx +1 #update index of spec df
    
    #pause after every 10 loops
    #change user agent, chosen randomly every 10 loops
    if idx%2 == 0:
        time.sleep(0.5+1*random.random())
        headers = {'user-agent': ua.random}

In [55]:
model_spec_df.to_pickle('../data/model_spec_df.pkl')
model_spec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7025 entries, 0 to 7024
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Model          7025 non-null   object
 1   url            7025 non-null   object
 2   drive          7022 non-null   object
 3   engine         5818 non-null   object
 4   price          5278 non-null   object
 5   doors          3999 non-null   object
 6   passengers     3999 non-null   object
 7   speed_sec      5590 non-null   object
 8   horsepower_hp  6946 non-null   object
 9   mpg            6416 non-null   object
 10  tank_gal       6879 non-null   object
 11  volume_cuft    1765 non-null   object
 12  length_in      5639 non-null   object
 13  width_in       5916 non-null   object
 14  height_in      5937 non-null   object
 15  wheelbase_in   5961 non-null   object
 16  Year           7025 non-null   object
dtypes: object(17)
memory usage: 933.1+ KB


## Clean Data

Remove symbols and units from data in df so data can be changed to numeric datatype

In [900]:
model_spec_df2 = model_spec_df.copy()
yearly_sales_df2 = yearly_sales_df.copy()
replace_list = ['hp', 'mpg', 'gal.', 'cu.ft.', 'in.', ',', 'sec', '$', '$']

#loop through strings to remove in df
for s in replace_list:
    model_spec_df2 = model_spec_df2.replace(s,'', regex=True) 
model_spec_df2['price'] = model_spec_df2['price'].str.replace('$','') #endure dollar sign is removed

In [932]:
model_spec_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7025 entries, 0 to 7024
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Model          7025 non-null   object 
 1   url            7025 non-null   object 
 2   drive          7022 non-null   object 
 3   engine         5818 non-null   object 
 4   price          5278 non-null   float64
 5   doors          3999 non-null   float64
 6   passengers     3999 non-null   float64
 7   speed_sec      5587 non-null   float64
 8   horsepower_hp  6946 non-null   float64
 9   mpg            6416 non-null   float64
 10  tank_gal       6879 non-null   float64
 11  volume_cuft    1765 non-null   float64
 12  length_in      5639 non-null   float64
 13  width_in       5916 non-null   float64
 14  height_in      5935 non-null   float64
 15  wheelbase_in   5961 non-null   float64
 16  Year           7025 non-null   object 
 17  cleaned        7025 non-null   object 
 18  cleaned_

Change all columns to numeric except for drive, engine, doors, and passengers

In [902]:
num_cols = ['price',  'speed_sec', 'horsepower_hp', 'mpg', 'tank_gal', 'volume_cuft', 
            'width_in', 'length_in','height_in','wheelbase_in']
for col in num_cols:
    model_spec_df2[col] = model_spec_df2[col].apply(pd.to_numeric, errors='coerce')

model_spec_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7025 entries, 0 to 7024
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Model          7025 non-null   object 
 1   url            7025 non-null   object 
 2   drive          7022 non-null   object 
 3   engine         5818 non-null   object 
 4   price          5278 non-null   float64
 5   doors          3999 non-null   float64
 6   passengers     3999 non-null   float64
 7   speed_sec      5587 non-null   float64
 8   horsepower_hp  6946 non-null   float64
 9   mpg            6416 non-null   float64
 10  tank_gal       6879 non-null   float64
 11  volume_cuft    1765 non-null   float64
 12  length_in      5639 non-null   float64
 13  width_in       5916 non-null   float64
 14  height_in      5935 non-null   float64
 15  wheelbase_in   5961 non-null   float64
 16  Year           7025 non-null   object 
dtypes: float64(12), object(5)
memory usage: 933.1+ KB


In [903]:
yearly_sales_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394 entries, 0 to 4516
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Model        4394 non-null   object
 1   Total_Sales  4311 non-null   object
 2   Year         4394 non-null   object
dtypes: object(3)
memory usage: 137.3+ KB


#### Left merge Sales df to specs df and find info

Clean Model Names in each Dataframe before merging

In [904]:
import re

def clean_sales_name(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', ' ', name)
#     name = name.replace('series', '')
    stopwords = ['series', 'matrix']
    word_list = name.split()
    word_list  = [word for word in word_list if word not in stopwords]
    name = ' '.join(word_list)
    return name.strip()

def clean_name(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', ' ', name)
   
    return name.strip()

def find_partial_match(name, name_list):
    if 'ford' in name or 'chevrolet' in name or 'gmc' in name:
        name = re.sub('[0-9\n]','',name)
    elif 'bmw' in name:
        name = name.replace('series', '')

    return name.strip()

In [905]:
model_spec_df2['cleaned'] = model_spec_df2.apply(lambda x: clean_name(x['Model']), axis=1)
yearly_sales_df2['cleaned_name'] = yearly_sales_df2.apply(lambda x: clean_sales_name(x['Model']), axis=1)


name_list = yearly_sales_df2.cleaned_name.unique()
model_spec_df2['cleaned_name'] = model_spec_df2.apply(lambda x: find_partial_match(x['cleaned'], name_list), axis=1)

yearly_sales_df2['Total_Sales'] = yearly_sales_df2['Total_Sales'].apply(pd.to_numeric, errors='coerce')

perform merge on Model and Year columns

In [906]:
model_spec_sales = yearly_sales_df2.merge(model_spec_df2, how = 'left',on=["cleaned_name",'Year'])
                                    
engine = model_spec_sales.groupby(['cleaned_name','Year'])['engine'].apply(lambda x: np.nan if x.isnull().all() else x.value_counts().index[0]).reset_index()
# engine = engine.apply(lambda y: float('NaN') if len(y)==0 else y)

drive = model_spec_sales.groupby(['cleaned_name','Year'])['drive'].apply(lambda x: np.nan if x.isnull().all() else x.value_counts().index[0]).reset_index()
drive = drive.apply(lambda y: float('NaN') if len(y)==0 else y)
 
model_spec_sales_df = model_spec_sales.groupby(['cleaned_name','Year']).mean().reset_index()
    
model_spec_sales_df['engine'] = engine['engine']
model_spec_sales_df['drive'] = drive['drive']

# http://www.carqueryapi.com/
# https://www.fueleconomy.gov/feg/download.shtml
# https://www.reddit.com/r/datasets/comments/b6rcwv/i_scraped_32000_cars_including_the_price_and_115/
model_spec_sales_df.info()
# print(model_spec_sales_df[2000:3200])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4394 entries, 0 to 4393
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cleaned_name   4394 non-null   object 
 1   Year           4394 non-null   object 
 2   Total_Sales    4311 non-null   float64
 3   price          2720 non-null   float64
 4   doors          2234 non-null   float64
 5   passengers     2234 non-null   float64
 6   speed_sec      2537 non-null   float64
 7   horsepower_hp  3155 non-null   float64
 8   mpg            3100 non-null   float64
 9   tank_gal       3129 non-null   float64
 10  volume_cuft    1018 non-null   float64
 11  length_in      2983 non-null   float64
 12  width_in       3095 non-null   float64
 13  height_in      3095 non-null   float64
 14  wheelbase_in   3109 non-null   float64
 15  engine         3035 non-null   object 
 16  drive          3183 non-null   object 
dtypes: float64(13), object(4)
memory usage: 583.7+ KB


We see that some of the cars with the top sales did not get any spec data. This needs to be further investigated to see if further merging could be done

Find columns total sales df and in specs df that did not merge

In [907]:
unique_sales_df = (yearly_sales_df2.merge(model_spec_df2, on='cleaned_name', how='outer', indicator=True)
            .query('_merge != "both"').drop(columns='_merge'))
unique_sales_df['cleaned_name'].value_counts().head(20) #find top rows that did not merge

gmc sierra hd                  25
chevrolet silverado hd         25
mazda mazda6                   18
mazda mazda3                   16
ford super duty f              16
nissan pathfer                 16
chevrolet equx                 16
loln navigator                 16
audi a8 l                      15
gmc savana passenger           15
bentley contntal gt            15
chevrolet express passenger    15
honda civic coupe              14
rolls royce phantom            14
audi s4                        14
loln mkz                       14
cadillac cts v                 14
honda accord sedan             14
gmc savana cargo               13
lincoln navigator              13
Name: cleaned_name, dtype: int64

Fill in null values, using previous values of the same model

In [909]:
model_spec_sales_df2 = model_spec_sales_df.groupby('cleaned_name').ffill().reindex(model_spec_sales_df.columns, axis=1)
model_spec_sales_df2['cleaned_name'] = model_spec_sales_df['cleaned_name']
# model_spec_sales_df2[2100:2150]
model_spec_sales_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4394 entries, 0 to 4393
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cleaned_name   4394 non-null   object 
 1   Year           4394 non-null   object 
 2   Total_Sales    4352 non-null   float64
 3   price          3494 non-null   float64
 4   doors          3012 non-null   float64
 5   passengers     3012 non-null   float64
 6   speed_sec      3301 non-null   float64
 7   horsepower_hp  3575 non-null   float64
 8   mpg            3556 non-null   float64
 9   tank_gal       3548 non-null   float64
 10  volume_cuft    1816 non-null   float64
 11  length_in      3502 non-null   float64
 12  width_in       3559 non-null   float64
 13  height_in      3559 non-null   float64
 14  wheelbase_in   3561 non-null   float64
 15  engine         3533 non-null   object 
 16  drive          3593 non-null   object 
dtypes: float64(13), object(4)
memory usage: 583.7+ KB


In [933]:
model_spec_sales_df2['engine'].value_counts()

Gas          3375
Electric       60
Flex Fuel      44
Hybrid         38
Diesel         16
Name: engine, dtype: int64

Even after filling null values there is a lot of missing spec data, maybe try finding more spec data

## Load more spec data

In [959]:
specs_df = pd.read_csv('/Users/racheldilley/Documents/car-sales-predictor-data/fullspecs.csv')

In [960]:
specs_df2 = specs_df.transpose()
specs_df2.columns = specs_df2.iloc[0]
specs_df2.drop(specs_df2.index[0], inplace=True)
specs_df3 = specs_df2[['MSRP','Gas Mileage', 'Engine', 'EPA Class', 'Drivetrain', 'Passenger Capacity', 'Passenger Doors',
                     'Body Style', 'Transmission','EPA Classification', 'Base Curb Weight (lbs)', 'Wheelbase (in)',
                      'Min Ground Clearance (in)', 'Height, Overall (in)', 'Fuel Tank Capacity, Approx (gal)',
                      'Fuel Economy Est-Combined (MPG)', 'Fuel System', 'Engine Type', 'Trans Description Cont.',
                      'Passenger Volume (ft³)', 'Length, Overall w/rear bumper (in)']].reset_index()
list(specs_df2.columns)

['MSRP',
 'Gas Mileage',
 'Engine',
 'EPA Class',
 'Style Name',
 'Drivetrain',
 'Passenger Capacity',
 'Passenger Doors',
 'Body Style',
 'Transmission',
 'EPA Classification',
 'Base Curb Weight (lbs)',
 'Front Hip Room (in)',
 'Front Leg Room (in)',
 'Second Shoulder Room (in)',
 'Passenger Volume (ft³)',
 'Second Head Room (in)',
 'Front Shoulder Room (in)',
 'Second Hip Room (in)',
 'Front Head Room (in)',
 'Second Leg Room (in)',
 'Wheelbase (in)',
 'Min Ground Clearance (in)',
 'Track Width, Front (in)',
 'Width, Max w/o mirrors (in)',
 'Track Width, Rear (in)',
 'Height, Overall (in)',
 'Cargo Volume to Seat 1 (ft³)',
 'Cargo Volume to Seat 2 (ft³)',
 'Cargo Volume to Seat 3 (ft³)',
 'Fuel Tank Capacity, Approx (gal)',
 'Fuel Economy Est-Combined (MPG)',
 'EPA Fuel Economy Est - City (MPG)',
 'EPA Fuel Economy Est - Hwy (MPG)',
 'Engine Order Code',
 'SAE Net Torque @ RPM',
 'Fuel System',
 'Engine Type',
 'SAE Net Horsepower @ RPM',
 'Displacement',
 'First Gear Ratio (:1)',
 

In [961]:
col_names = [ 'name','MSRP','mpg', 'engine', 'class', 'drivetrain', 'capacity', 'doors',
                     'body_style', 'transmission','EPA_class', 'curb_weight_lbs', 'wheelbase_in',
                      'ground_clearance_in', 'height_in', 'fuel_tank_cap_gal',
                      'combined_mpg', 'fuel_system', 'engine_type', 'trans_descr',
                      'passenger_volume_cubft', 'length_in']
specs_df3.columns = col_names
specs_df3.head()

,name,MSRP,mpg,engine,class,drivetrain,capacity,doors,body_style,transmission,...,wheelbase_in,ground_clearance_in,height_in,fuel_tank_cap_gal,combined_mpg,fuel_system,engine_type,trans_descr,passenger_volume_cubft,length_in
0,2019 Acura RDX Specs: FWD w/Technology Pkg,"$40,600",22 mpg City/28 mpg Hwy,"Turbo Premium Unleaded I-4, 2.0 L",Small Sport Utility Vehicles 2WD,Front Wheel Drive,5,4,Sport Utility,Transmission: 10-Speed Automatic -inc: sequent...,...,108.3,5.7,65.7,17.1,24.0,Gasoline Direct Injection,Turbo Premium Unleaded I-4,Automatic w/OD,104,NaN
1,2019 Acura RDX Specs: FWD w/Advance Pkg,"$45,500",22 mpg City/28 mpg Hwy,"Turbo Premium Unleaded I-4, 2.0 L",Small Sport Utility Vehicles 2WD,Front Wheel Drive,5,4,Sport Utility,Transmission: 10-Speed Automatic -inc: sequent...,...,108.3,5.7,65.7,17.1,24.0,Gasoline Direct Injection,Turbo Premium Unleaded I-4,Automatic w/OD,104,NaN
2,2019 Acura RDX Specs: FWD w/A-Spec Pkg,"$43,600",22 mpg City/27 mpg Hwy,"Turbo Premium Unleaded I-4, 2.0 L",Small Sport Utility Vehicles 2WD,Front Wheel Drive,5,4,Sport Utility,Transmission: 10-Speed Automatic -inc: sequent...,...,108.3,5.7,65.7,17.1,24.0,Gasoline Direct Injection,Turbo Premium Unleaded I-4,Automatic w/OD,104,NaN
3,2019 Acura RDX Specs: FWD,"$37,400",22 mpg City/28 mpg Hwy,"Turbo Premium Unleaded I-4, 2.0 L",Small Sport Utility Vehicles 2WD,Front Wheel Drive,5,4,Sport Utility,Transmission: 10-Speed Automatic -inc: sequent...,...,108.3,5.7,65.7,17.1,24.0,Gasoline Direct Injection,Turbo Premium Unleaded I-4,Automatic w/OD,104,NaN
4,2019 Acura RDX Specs: AWD w/Technology Pkg,"$42,600",21 mpg City/27 mpg Hwy,"Turbo Premium Unleaded I-4, 2.0 L",Small Sport Utility Vehicles 4WD,All Wheel Drive,5,4,Sport Utility,Transmission: 10-Speed Automatic -inc: sequent...,...,108.3,5.7,65.7,17.1,23.0,Gasoline Direct Injection,Turbo Premium Unleaded I-4,Automatic w/OD,104,NaN


### Clean new spec data

Clean car name

In [962]:
def get_year(name):
    year = name[0:4]
    return year.strip()

def get_name(name):
    name = name[5:]
    sep = 'Specs:'
    stripped = name.split(sep, 1)[0]
    name = stripped.replace(' Specs:', '')
    return name

specs_df3['Year'] = specs_df3.apply(lambda x: get_year(x['name']), axis=1)
specs_df3['Model'] = specs_df3.apply(lambda x: get_name(x['name']), axis=1)
specs_df3.drop('name', axis=1, inplace=True)
specs_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32316 entries, 0 to 32315
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   MSRP                    32262 non-null  object
 1   mpg                     26292 non-null  object
 2   engine                  30341 non-null  object
 3   class                   30340 non-null  object
 4   drivetrain              30600 non-null  object
 5   capacity                32316 non-null  object
 6   doors                   32316 non-null  object
 7   body_style              30600 non-null  object
 8   transmission            29602 non-null  object
 9   EPA_class               19918 non-null  object
 10  curb_weight_lbs         19707 non-null  object
 11  wheelbase_in            30345 non-null  object
 12  ground_clearance_in     17826 non-null  object
 13  height_in               30284 non-null  object
 14  fuel_tank_cap_gal       30179 non-null  object
 15  co

In [963]:
# remove characters before chaning columns to numeric
specs_df3['MSRP'] = specs_df3['MSRP'].str.replace('$','') 
specs_df3['MSRP'] = specs_df3['MSRP'].str.replace(',','') 

#change columns to numeric
num_cols = [ 'MSRP',  'capacity', 'doors','curb_weight_lbs', 'wheelbase_in','ground_clearance_in', 'height_in',
            'fuel_tank_cap_gal','combined_mpg', 'passenger_volume_cubft', 'length_in']

for col in num_cols:
    specs_df3[col] = specs_df3[col].apply(pd.to_numeric, errors='coerce')

In [964]:
#aggregate specs df
str_cols = [ 'mpg','engine', 'class', 'drivetrain', 'body_style', 'transmission','EPA_class', 
                 'fuel_system', 'engine_type', 'trans_descr',]
specs_df4 = specs_df3.groupby(['Model','Year']).mean().reset_index()

# add non-numerical columns with model values
for col in str_cols:

    df_feat = specs_df3.groupby(['Model','Year'])[col].apply(lambda x: np.nan if x.isnull().all() else x.value_counts().index[0]).reset_index()

    specs_df4[col] = df_feat[col]

# add 2020 data using 2019 data
df_2020 = specs_df4[specs_df4['Year'] == '2019']
df_2020['Year'] = df_2020['Year'].str.replace('2019','2020') 
specs_df4 = pd.concat([specs_df4, df_2020], ignore_index = True)


# add 2017 data using 2016 data
df_2017 = specs_df4[specs_df4['Year'] == '2016']
df_2017['Year'] = df_2017['Year'].str.replace('2016','2017') 
specs_df4 = pd.concat([specs_df4, df_2017], ignore_index = True)

In [965]:
specs_df4[400:430]

,Model,Year,MSRP,capacity,doors,curb_weight_lbs,wheelbase_in,ground_clearance_in,height_in,fuel_tank_cap_gal,...,mpg,engine,class,drivetrain,body_style,transmission,EPA_class,fuel_system,engine_type,trans_descr
400,BMW 7-Series,1990,58266.666667,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,BMW 7-Series,1991,62400.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,BMW 7-Series,1992,64013.333333,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Elect controlled 4-speed automatic transmissio...,NaN,NaN,NaN,NaN
403,BMW 7-Series,1993,66166.666667,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Elect controlled 5-speed automatic transmissio...,NaN,NaN,NaN,NaN
404,BMW 7-Series,1994,68283.333333,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Electronically controlled 5-speed automatic tr...,NaN,NaN,NaN,NaN
405,BMW 7-Series,1995,70566.666667,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Electronically controlled 5-speed automatic tr...,NaN,NaN,NaN,NaN
406,BMW 7-Series,1996,62490.000000,0.000000,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Rear Wheel Drive,4dr Car,5-speed electronically-controlled automatic tr...,NaN,NaN,NaN,NaN
407,BMW 7-Series,1997,72250.000000,5.000000,4.0,4365.333333,119.066667,NaN,56.233333,23.366667,...,17 mpg City/24 mpg Hwy,"Gas V8, 4.4L",Large,Rear Wheel Drive,4dr Car,5-speed electronically-controlled automatic tr...,Large,EFI,Gas V8,"Automatic w/OD,"
408,BMW 7-Series,1998,73033.333333,5.000000,4.0,4365.333333,119.066667,NaN,56.233333,23.366667,...,17 mpg City/24 mpg Hwy,"Gas V8, 4.4L",Large,Rear Wheel Drive,4dr Car,5-speed electronically-controlled automatic tr...,Large,EFI,Gas V8,"Automatic w/OD,"
409,BMW 7-Series,1999,73633.333333,5.000000,4.0,4365.333333,119.066667,NaN,56.233333,23.366667,...,17 mpg City/23 mpg Hwy,"Gas V8, 4.4L",Large,Rear Wheel Drive,4dr Car,5-speed electronically-controlled automatic tr...,Large,EFI,Gas V8,"Automatic w/OD,"


In [966]:
# clean name before merge with sales df
specs_df4['cleaned'] = specs_df4.apply(lambda x: clean_name(x['Model']), axis=1)

name_list = yearly_sales_df2.cleaned_name.unique()
specs_df4['cleaned_name'] = specs_df4.apply(lambda x: find_partial_match(x['cleaned'], name_list), axis=1)

#save as csv to manually add in missing data
specs_df4.to_csv('../Data/full_specs.csv')

# merge
new_model_spec_sales = yearly_sales_df2.merge(specs_df4, how = 'left',on=["cleaned_name",'Year'])
new_model_spec_sales.drop('Model_y', axis=1, inplace=True)

In [967]:
new_model_spec_sales.drop_duplicates(['cleaned_name', 'Year'], keep='first', inplace=True)

# fill in null values 
new_model_spec_sales2 = new_model_spec_sales.groupby('cleaned_name').ffill().reindex(new_model_spec_sales.columns, axis=1)
new_model_spec_sales2['cleaned_name'] = new_model_spec_sales['cleaned_name']

new_model_spec_sales2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394 entries, 0 to 4393
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Model_x                 4394 non-null   object 
 1   Total_Sales             4352 non-null   float64
 2   Year                    4394 non-null   object 
 3   cleaned_name            4394 non-null   object 
 4   MSRP                    2377 non-null   float64
 5   capacity                2377 non-null   float64
 6   doors                   2377 non-null   float64
 7   curb_weight_lbs         2127 non-null   float64
 8   wheelbase_in            2377 non-null   float64
 9   ground_clearance_in     1694 non-null   float64
 10  height_in               2377 non-null   float64
 11  fuel_tank_cap_gal       2353 non-null   float64
 12  combined_mpg            1875 non-null   float64
 13  passenger_volume_cubft  1954 non-null   float64
 14  length_in               118 non-null    

In [968]:
unique_sales_df = (yearly_sales_df2.merge(specs_df4, on='cleaned_name', how='outer', indicator=True)
            .query('_merge != "both"').drop(columns='_merge'))
unique_sales_df['cleaned_name'].value_counts().head(20) #find top rows that did not merge

honda accord sedan             26
gmc savana passenger           25
audi s4                        23
ford super duty f              21
gmc sierra hd                  18
chevrolet express passenger    18
mazda mazda6                   18
chevrolet silverado hd         18
audi a8 l                      17
rolls royce phantom            17
honda civic coupe              16
mazda 3                        16
audi a8                        16
mazda 6                        16
cadillac cts v                 16
honda accord                   15
mazda mazda3                   14
lincoln navigator l            14
audi r8                        14
dodge viper                    14
Name: cleaned_name, dtype: int64

### Combine both dataframes

In [969]:
partial_model_spec_sales_df = new_model_spec_sales2.merge(model_spec_sales_df2, how = 'left',on=["cleaned_name",
                                                                                                 'Year', 'Total_Sales'])

partial_model_spec_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394 entries, 0 to 4393
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Model_x                 4394 non-null   object 
 1   Total_Sales             4352 non-null   float64
 2   Year                    4394 non-null   object 
 3   cleaned_name            4394 non-null   object 
 4   MSRP                    2377 non-null   float64
 5   capacity                2377 non-null   float64
 6   doors_x                 2377 non-null   float64
 7   curb_weight_lbs         2127 non-null   float64
 8   wheelbase_in_x          2377 non-null   float64
 9   ground_clearance_in     1694 non-null   float64
 10  height_in_x             2377 non-null   float64
 11  fuel_tank_cap_gal       2353 non-null   float64
 12  combined_mpg            1875 non-null   float64
 13  passenger_volume_cubft  1954 non-null   float64
 14  length_in_x             118 non-null    

In [970]:
import math
def fill_null_values(x, y):
    try:
        if math.isnan(x):
            return y
        else:
            return x
    except:
        return x

drop_columns = ['Model_x', 'cleaned', 'mpg_x']
partial_model_spec_sales_df2 = partial_model_spec_sales_df.copy()
partial_model_spec_sales_df2.drop(drop_columns, axis=1, inplace=True )

column_pairs = {'msrp': ['MSRP','price'], 'pass_capacity': ['capacity', 'passengers'], 'doors':['doors_x', 'doors_y'],
               'wheelbase_in': ['wheelbase_in_x', 'wheelbase_in_y'], 'height_in': ['height_in_x', 'height_in_y'],
               'tank_cap_gal': ['fuel_tank_cap_gal', 'tank_gal'], 'comb_mpg': ['combined_mpg', 'mpg_y'],
               'engine':['engine_x', 'engine_y'], 'drive_train': ['drivetrain', 'drive'] , 'inside_vol_cuft': 
               ['passenger_volume_cubft', 'volume_cuft'], 'length_in': ['length_in_y', 'length_in_x']}

for key, value in column_pairs.items():
    partial_model_spec_sales_df2[key] = partial_model_spec_sales_df2.apply(lambda x: fill_null_values(x[value[0]], x[value[1]]), axis=1)
    partial_model_spec_sales_df2.drop(value, axis=1, inplace=True)

In [971]:
#remove rowws with nan total sales
partial_model_spec_sales_df2 = partial_model_spec_sales_df2[partial_model_spec_sales_df2['Total_Sales'].notna()]

partial_model_spec_sales_df2 = partial_model_spec_sales_df2.replace('No Data', float('NaN'), regex=True)
partial_model_spec_sales_df2.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 4352 entries, 0 to 4393
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Total_Sales          4352 non-null   float64
 1   Year                 4352 non-null   object 
 2   cleaned_name         4352 non-null   object 
 3   curb_weight_lbs      2127 non-null   float64
 4   ground_clearance_in  1694 non-null   float64
 5   class                2065 non-null   object 
 6   body_style           2377 non-null   object 
 7   transmission         2290 non-null   object 
 8   EPA_class            2128 non-null   object 
 9   fuel_system          2349 non-null   object 
 10  engine_type          2370 non-null   object 
 11  trans_descr          2377 non-null   object 
 12  speed_sec            3301 non-null   float64
 13  horsepower_hp        3575 non-null   float64
 14  width_in             3559 non-null   float64
 15  msrp                 3690 non-null   f

### Load more spec data (EPA Data)

In [996]:
specs_df = pd.read_csv('/Users/racheldilley/Documents/car-sales-predictor-data/vehicles.csv')
epa_specs_df = specs_df[['year', 'make', 'model', 'comb08', 'cylinders', 'drive',
                        'eng_dscr', 'fuelType', 'hlv', 'hpv', 'lv2', 'lv4', 'pv2', 'pv4', 'trany', 'VClass']]
epa_specs_df = epa_specs_df[epa_specs_df['year'] >= 2005]

In [997]:
epa_specs_df['lug_vol_cuft'] = epa_specs_df.apply(lambda x: x['hlv'] + x['lv2'] + x['lv4'], axis=1)
epa_specs_df['pass_vol_cuft'] = epa_specs_df.apply(lambda x: x['hpv'] + x['pv2'] + x['pv4'], axis=1)

# clean name and year before merge with sales df
epa_specs_df['Year'] = epa_specs_df.apply(lambda x: str(x['year']), axis=1)
epa_specs_df['name'] = epa_specs_df.apply(lambda x: x['make'] + ' ' + x['model'], axis=1)
epa_specs_df['cleaned'] = epa_specs_df.apply(lambda x: clean_name(x['name']), axis=1)
epa_specs_df['cleaned_name'] = epa_specs_df.apply(lambda x: find_partial_match(x['cleaned'], name_list), axis=1)


drop_cols = ['name', 'cleaned', 'hlv', 'hpv', 'lv2', 'lv4', 'pv2', 'pv4', 'year', 'make', 'model' ]
epa_specs_df2 = epa_specs_df.copy()
epa_specs_df2.drop(drop_cols, axis=1, inplace=True)
epa_specs_df2.tail()

,comb08,cylinders,drive,eng_dscr,fuelType,trany,VClass,lug_vol_cuft,pass_vol_cuft,Year,cleaned_name
37238,14,8.0,Rear-Wheel Drive,SIDI,Regular,Automatic 6-spd,Special Purpose Vehicle 2WD,0,0,2021,chevrolet silverado cab chassis wd
37239,14,8.0,4-Wheel Drive,SIDI,Regular,Automatic 6-spd,Special Purpose Vehicle 4WD,0,0,2021,gmc sierra cab chassis wd
37240,14,8.0,Rear-Wheel Drive,SIDI,Regular,Automatic 6-spd,Special Purpose Vehicle 2WD,0,0,2021,gmc sierra cab chassis wd
37241,133,NaN,Front-Wheel Drive,NaN,Electricity,Automatic (A1),Midsize Cars,23,96,2021,hyundai ioniq electric
37243,129,NaN,Rear-Wheel Drive,NaN,Electricity,Automatic (A1),Small Sport Utility Vehicle 2WD,0,0,2021,tesla model y standard range rwd


In [998]:
epa_specs_df2['fuelType'].value_counts()

Regular                        9542
Premium                        8765
Gasoline or E85                1221
Diesel                          328
Electricity                     256
Midgrade                        130
Premium or E85                  122
Premium and Electricity         100
Regular Gas and Electricity      63
Premium Gas or Electricity       57
CNG                              18
Regular Gas or Electricity        4
Gasoline or natural gas           2
Name: fuelType, dtype: int64

In [999]:
# merge dataframes
partial_model_spec_sales_df3 = partial_model_spec_sales_df2.merge(epa_specs_df2, how = 'left',on=["cleaned_name",'Year'])
partial_model_spec_sales_df3.drop_duplicates(['cleaned_name', 'Year'], keep='first', inplace=True)
# partial_model_spec_sales_df3.info()

In [1000]:
column_pairs = {'vol_cubft': ['pass_vol_cuft', 'inside_vol_cuft'], 'mpg_comb': ['comb08', 'comb_mpg'], 
               'drivetrain': ['drive_train', 'drive'], 'fuel_typ': ['engine', 'fuelType'],
               'trans': ['transmission', 'trany'], 'class_EPA': ['EPA_class', 'VClass'], 'engine':
                ['fuel_system', 'eng_dscr']
               }

for key, value in column_pairs.items():
    partial_model_spec_sales_df3[key] = partial_model_spec_sales_df3.apply(lambda x: fill_null_values(x[value[0]], x[value[1]]), axis=1)
    partial_model_spec_sales_df3.drop(value, axis=1, inplace=True)

drop_cols = ['trans_descr', 'class']
partial_model_spec_sales_df4 = partial_model_spec_sales_df3.copy()
partial_model_spec_sales_df4.drop(drop_cols, axis=1, inplace=True)

partial_model_spec_sales_df5 = partial_model_spec_sales_df4.groupby('cleaned_name').ffill().reindex(partial_model_spec_sales_df3.columns, axis=1)
partial_model_spec_sales_df5['cleaned_name'] = partial_model_spec_sales_df4['cleaned_name']

partial_model_spec_sales_df5 = partial_model_spec_sales_df5.sort_values(['Year'], ascending=False)

partial_model_spec_sales_df5 = partial_model_spec_sales_df4.groupby('cleaned_name').ffill().reindex(partial_model_spec_sales_df3.columns, axis=1)
partial_model_spec_sales_df5['cleaned_name'] = partial_model_spec_sales_df4['cleaned_name']


In [1001]:
#save as csv to manually add in missing data
partial_model_spec_sales_df5.to_csv('../Data/partial_model_spec_sales_df.csv')

partial_model_spec_sales_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4352 entries, 0 to 6549
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Total_Sales          4352 non-null   float64
 1   Year                 4352 non-null   object 
 2   cleaned_name         4352 non-null   object 
 3   curb_weight_lbs      2127 non-null   float64
 4   ground_clearance_in  1694 non-null   float64
 5   body_style           2377 non-null   object 
 6   engine_type          2370 non-null   object 
 7   speed_sec            3301 non-null   float64
 8   horsepower_hp        3575 non-null   float64
 9   width_in             3559 non-null   float64
 10  msrp                 3690 non-null   float64
 11  pass_capacity        3401 non-null   float64
 12  doors                3401 non-null   float64
 13  wheelbase_in         3702 non-null   float64
 14  height_in            3700 non-null   float64
 15  tank_cap_gal         3690 non-null   f

In [987]:
null_df = full_model_spec_sales_df[full_model_spec_sales_df[null_cols].isna().all(1)] #.groupby('cleaned_name')
null_df['cleaned_name'].value_counts()
# pd.DataFrame(null_df)

chrysler town country    10
infiniti fx               8
infiniti qx56             8
infiniti m                7
mercedes benz slk         7
                         ..
hyundai genesis ³         1
buick encore gx           1
chevrolet tracker         1
jeep cherokee             1
toyota prius prime        1
Name: cleaned_name, Length: 291, dtype: int64

In [988]:
yearly_sales_df2[yearly_sales_df2['cleaned_name'].str.contains('fcx')]

,Model,Total_Sales,Year,cleaned_name
294,Honda FCX,6.0,2005,honda fcx
611,Honda FCX,6.0,2006,honda fcx
921,Honda FCX,10.0,2007,honda fcx
1214,Honda FCX,11.0,2008,honda fcx
1511,Honda FCX,5.0,2009,honda fcx
1795,Honda FCX,17.0,2010,honda fcx
2071,Honda FCX,2.0,2011,honda fcx
2374,Honda FCX,5.0,2012,honda fcx
2666,Honda FCX,10.0,2013,honda fcx
2902,Honda FCX,2.0,2014,honda fcx
